# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
J. Li  ->  J. Li  |  ['J. Li']
J. Li  ->  J. Li  |  ['J. Li']
I. J. M. Crossfield  ->  I. J. M. Crossfield  |  ['I. J. M. Crossfield']
K. Lee  ->  K. Lee  |  ['K. Lee']
F. Walter  ->  F. Walter  |  ['F. Walter']


H. Beuther  ->  H. Beuther  |  ['H. Beuther']
S. Li  ->  S. Li  |  ['S. Li']
Arxiv has 66 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2408.09051


extracting tarball to tmp_2408.09051...

 done.
Retrieving document from  https://arxiv.org/e-print/2408.09216
extracting tarball to tmp_2408.09216...

/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2723/1211882699.py:51: LatexWarning: 2408.09051 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


...

Retrieving document from  https://arxiv.org/e-print/2408.10199


extracting tarball to tmp_2408.10199...

 done.


H. Beuther  ->  H. Beuther  |  ['H. Beuther']
S. Li  ->  S. Li  |  ['S. Li']


Found 41 bibliographic references in tmp_2408.10199/ms2.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.09216-b31b1b.svg)](https://arxiv.org/abs/2408.09216) | **Asymmetric dust accumulation of the PDS 70 disk revealed by ALMA Band 3 observations**  |
|| K. Doi, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2024-08-20*|
|*Comments*| *submitted*|
|**Abstract**|            The PDS 70 system, hosting two planets within its disk, is an ideal target for examining the effect of planets on dust accumulation, growth, and ongoing planet formation. Here, we present high-resolution ($0.''07 = 8 \ \mathrm{au}$) dust continuum observations of the PDS 70 disk in ALMA Band 3 (3.0 mm). While previous Band 7 observations showed a dust ring with slight asymmetry, our Band 3 observations reveal a more prominent asymmetric peak in the northwest direction, where the intensity is 2.5 times higher than in other directions and the spectral index is at the local minimum with $\alpha_{\mathrm{SED}} \sim 2.2$. This indicates that a substantial amount of dust is accumulated both radially and azimuthally in the peak. We also detect point-source emission around the stellar position in the Band 3 image, which is likely to be free-free emission. We constrain the eccentricity of the outer ring to be $e<0.04$ from the position of the central star and the outer ring. From the comparison with numerical simulations, we constrain the mass of PDS 70c to be less than 4.9 Jupiter masses if the gas turbulence strength $\alpha_{\mathrm{turb}} = 10^{-3}$. Then, we discuss the formation mechanism of the disk structures and further planet formation scenarios in the disk.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.10199-b31b1b.svg)](https://arxiv.org/abs/2408.10199) | **Magnetic Fields in Massive Star-forming Regions (MagMaR) IV: Tracing the Magnetic Fields in the O-type protostellar system IRAS 16547$-$4247**  |
|| L. A. Zapata, et al. -- incl., <mark>H. Beuther</mark>, <mark>S. Li</mark> |
|*Appeared on*| *2024-08-20*|
|*Comments*| *Accepted by the Astrophysical Journal, 13 pages*|
|**Abstract**|            The formation of the massive stars, and in particular, the role that the magnetic fields play in their early evolutionary phase is still far from being completely understood. Here, we present Atacama Large Millimeter/Submillimeter Array (ALMA) 1.2 mm full polarized continuum, and H$^{13}$CO$^+$(3$-$2), CS(5$-$4), and HN$^{13}$C(3$-$2) line observations with a high angular resolution ($\sim$0.4$''$ or 1100 au). In the 1.2 mm continuum emission, we reveal a dusty envelope surrounding the massive protostars, IRAS16547-E and IRAS16547-W, with dimensions of $\sim$10,000 au. This envelope has a bi-conical structure likely carved by the powerful thermal radio jet present in region. The magnetic fields vectors follow very-well the bi-conical envelope. The polarization fraction is $\sim$2.0\% in this region. Some of these vectors seem to converge to IRAS 16547-E, and IRAS 16547-W, the most massive protostars. Moreover, the velocity fields revealed from the spectral lines H$^{13}$CO$^+$(3$-$2), and HN$^{13}$C(3$-$2) show velocity gradients with a good correspondence with the magnetic fields, that maybe are tracing the cavities of molecular outflows or maybe in some parts infall. We derived a magnetic field strength in some filamentary regions that goes from 2 to 6.1\,mG. We also find that the CS(5$-$4) molecular line emission reveals multiple outflow cavities or bow-shocks with different orientations, some of which seem to follow the NW-SE radio thermal jet.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.09502-b31b1b.svg)](https://arxiv.org/abs/2408.09502) | **The Electrical Design of a Membrane Antenna for Lunar-based Low-frequency Radio Telescope**  |
|| Suonanben, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-08-20*|
|*Comments*| *14 pages, 19 figures*|
|**Abstract**|            Detecting primordial fluctuations from the cosmic dark ages requires extremely large low-frequency radio telescope arrays deployed on the far side of the Moon. The antenna of such an array must be lightweight, easily storable and transportable, deployable on a large scale, durable, and capable of good electrical performance. A membrane antenna is an excellent candidate to meet these criteria. We study the design of a low-frequency membrane antenna for a lunar-based low-frequency (<30 MHz) radio telescope constructed from polyimide film widely used in aerospace applications, owing to its excellent dielectric properties and high stability as a substrate material. We first design and optimize an antenna in free space through dipole deformation and coupling principles, then simulate an antenna on the lunar surface with a simple lunar soil model, yielding an efficiency greater than 90% in the range of 12-19 MHz and greater than 10% in the range of 5-35 MHz. The antenna inherits the omni-directional radiation pattern of a simple dipole antenna in the 5-30 MHz frequency band, giving a large field of view and allowing detection of the 21 cm global signal when used alone. A demonstration prototype is constructed, and its measured electrical property is found to be consistent with simulated results using |S11| measurements. This membrane antenna can potentially fulfill the requirements of a lunar low-frequency array, establishing a solid technical foundation for future large-scale arrays for exploring the cosmic dark ages.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.09606-b31b1b.svg)](https://arxiv.org/abs/2408.09606) | **Global weather map reveals persistent top-of-atmosphere features on the nearest brown dwarfs**  |
|| X. Chen, et al. -- incl., <mark>I. J. M. Crossfield</mark> |
|*Appeared on*| *2024-08-20*|
|*Comments*| *32 pages, 14 figures, accepted for publication in MNRAS*|
|**Abstract**|            Brown dwarfs and planetary-mass companions display rotationally modulated photometric variability, especially those near the L/T transition. This variability is commonly attributed to top-of-atmosphere (TOA) inhomogeneities, with proposed models including patchy thick and thin clouds, planetary-scale jets, or chemical disequilibrium. Surface mapping techniques are powerful tools to probe their atmospheric structures and distinguish between models. One of the most successful methods for stellar surface mapping is Doppler imaging, where the existence of TOA inhomogeneities can be inferred from their varying Doppler shifts across the face of a rotating star. We applied Doppler imaging to the nearest brown dwarf binary WISE 1049AB (aka Luhman 16AB) using time-resolved, high-resolution spectroscopic observations from Gemini IGRINS, and obtained for the first time H and K band simultaneous global weather map for brown dwarfs. Compared to the only previous Doppler map for a brown dwarf in 2014 featuring a predominant mid-latitude cold spot on WISE 1049B and no feature on WISE 1049A, our observations detected persistent spot-like structures on WISE 1049B in the equatorial to mid-latitude regions on two nights, and revealed new polar spots on WISE 1049A. Our results suggest stability of atmospheric features over timescale of days and possible long-term stable or recurring structures. H and K band maps displayed similar structures in and out of CO bands, indicating the cold spots not solely due to chemical hotspots but must involve clouds. Upcoming 30-m extremely large telescopes (ELTs) will enable more sensitive Doppler imaging of dozens of brown dwarfs and even a small number of directly-imaged exoplanets.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.09708-b31b1b.svg)](https://arxiv.org/abs/2408.09708) | **AXPs/SGRs: strange stars with crusts?**  |
|| G. Qiao, et al. -- incl., <mark>K. Lee</mark> |
|*Appeared on*| *2024-08-20*|
|*Comments*| *12 pages, 3 figures, accepted for publication in JUSTC*|
|**Abstract**|            The emission of Anomalous X-ray Pulsars (AXPs) and Soft Gamma-Ray Repeaters (SGRs) is believed to be powered by the dissipation of their strong magnetic fields, which coined the name `magnetar'. By combining timing and energy observational results, the magnetar model can be easily appreciated. From a timing perspective, the magnetic field strengths of AXPs and SGRs, calculated assuming dipole radiation, are extremely strong. From an energy perspective, the X-ray/soft gamma-ray luminosities of AXPs and SGRs are larger than their rotational energy loss rates (i.e., $ L_{\rm X}>\dot E_{\rm rot}$). It is thus reasonable to assume that the high-energy radiation comes from magnetic energy decay, and the magnetar model has been extensively discussed (or accepted). However, we argue that: (1) calculating magnetic fields by assuming that rotational energy loss is dominated by dipole radiation (i.e., $\dot{E}_{\rm rot}\simeq\dot{E}_{\mu}$)) may be controversial, and we suggest that the energies carried by outflowing particles should also be considered; and (2) the fact that X-ray luminosity is greater than the rotational energy loss rate does not necessarily mean that the emission energy comes from the magnetic field decaying, which requires further observational testing. Furthermore, some observational facts conflict with the `magnetar' model, such as observations of anti-magnetars, high magnetic field pulsars, and radio and X-ray observations of AXPs/SGRs. Therefore, we propose a crusted strange star model as an alternative, which can explain many more observational facts of AXPs/SGRs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.09051-b31b1b.svg)](https://arxiv.org/abs/2408.09051) | **AI-assisted super-resolution cosmological simulations IV: An emulator for deterministic realizations**  |
|| <mark>X. Zhang</mark>, et al. |
|*Appeared on*| *2024-08-20*|
|*Comments*| *16 pages, 15 figures*|
|**Abstract**|            Super-resolution (SR) models in cosmological simulations use deep learning (DL) to rapidly supplement low-resolution (LR) runs with statistically correct, fine details. The SR technique preserves large-scale structures by conditioning on a low-resolution (LR) version of the simulation. On smaller scales, the generative deep learning (DL) process is stochastic, resulting in numerous possible SR realizations, each with unique small-scale structures. Validation of SR then relies on making sure that a specific statistic of interest is accurately reproduced by comparing SR and high resolution (HR) runs. In this study, we develop an emulator designed to reproduce the individual small-scale structures of an HR simulation as closely as possible. We process an SR realization alongside a specific High-Resolution Initial Condition (HRIC), transforming the SR output to emulate the results of a full simulation with that HRIC. By comparing visualizations, individual halo measures and cross-correlating Fourier modes we show that the emulated SR runs closely align with the corresponding HR simulation, even on length scales an order of magnitude smaller than the LR run. Additionally, small halos are trained to match the HR simulation, and the subhalo mass function is more accurately reproduced. These results show the promise of this method for generating numerous fast and accurate simulations and mock observations for large galaxy surveys.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.09944-b31b1b.svg)](https://arxiv.org/abs/2408.09944) | **Detection of the [O I] 63 $\mu$m emission line from the $z = 6.04$ quasar J2054-0005**  |
|| N. Ishii, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2024-08-20*|
|*Comments*| *Submitted to PASJ. 5 figures and 3 tables. Comments are very welcome*|
|**Abstract**|            We report the highest-redshift detection of [O I] 63 $\mu$m from a luminous quasar, J2054-0005, at $z=6.04$ based on the Atacama Large Millimeter/sub-millimeter Array Band 9 observations. The [O I] 63 $\mu$m line luminosity is $(4.5\pm1.5) \times 10^{9}~L_{\rm \odot}$, corresponding to the [O I] 63 $\mu$m-to-far-infrared luminosity ratio of $\approx 6.7\times10^{-4}$, which is consistent with the value obtained in the local universe. Remarkably, [O I] 63 $\mu$m is as bright as [C II] 158 $\mu$m, resulting in the [O I]-to-[C II] line luminosity ratio of $1.3\pm0.5$. Based on a careful comparison of the luminosity ratios of [O I] 63 $\mu$m, [C II] 158 $\mu$m, and dust continuum emission to models of photo-dissociation regions, we find that J2054-0005 has a gas density log($n_{\rm H}$/cm$^{-3}$)$=3.7\pm0.3$ and an incident far-ultraviolet radiation field of log($G/G_{\rm 0}$)$= 3.0\pm0.1$, showing that [O I] 63 $\mu$m serves as an important coolant of the dense and warm gas in J2054-0005. A close examination of the [O I] and [C II] line profiles suggests that the [O I] line may be partially self-absorbed, however deeper observations are needed to verify this conclusion. Regardless, the gas density and incident radiation field are in a broad agreement with the values obtained in nearby star-forming galaxies and objects with [O I] 63 $\mu$m observations at $z=1-3$ with the Herschel Space Observatory. These results demonstrate the power of ALMA high-frequency observations targeting [O I] 63 $\mu$m to examine the properties of photo-dissociation regions in high-redshift galaxies.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Unable to locate Ghostscript on paths</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.09260-b31b1b.svg)](https://arxiv.org/abs/2408.09260) | **Analysis of the Effect of Tilted Corner Cube Reflector Arrays on Lunar Laser Ranging**  |
|| J. Cao, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-08-20*|
|*Comments*| **|
|**Abstract**|            This paper primarily investigates the effect of the tilt of corner cube reflector (CCR) arrays on lunar laser ranging (LLR). A mathematical model was established to study the random errors caused by the tilt of the CCR arrays. The study found that, ideally, when the laser ranging pulse width is 10 picoseconds or less, it is possible to distinguish from which specific corner cubes within the CCR array each peak in the echo signal originates. Consequently, partial data from the echo can be extracted for signal processing, significantly reducing random errors and improving the single-shot precision of LLR. The distance obtained by extracting part of the echo can be reduced to the center position of the array, thereby providing multiple higher-precision ranging results from each measurement. This not only improves the precision of LLR but also increases the data volume. A simulation experiment based on the 1.2 m laser ranging system at Yunnan Observatories was conducted. By extracting one peak for signal processing, the single-shot precision improved from 32.24 mm to 2.52 mm, validating the theoretical analysis results. Finally, an experimental laser ranging system based on a 53 cm binocular telescope system was established for ground experiments. The experimental results indicated that the echo signal could identify the tilt state of the CCR array. By extracting the peak returned by the central CCR for signal processing, the ranging precision was greatly improved. Through theoretical analyses, simulation experiments, and ground experiments, a solution to reduce the random errors caused by the tilt of the CCR array was provided. This offers an approach to enhance the single-shot precision of future LLR and provides a reference for upgrading ground-based equipment at future laser ranging stations.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2408.09216/./spectral_index_1x2.png', 'tmp_2408.09216/./ring_corner_B3.png', 'tmp_2408.09216/./delta_center_corner_B3.png']
copying  tmp_2408.09216/./spectral_index_1x2.png to _build/html/
copying  tmp_2408.09216/./ring_corner_B3.png to _build/html/
copying  tmp_2408.09216/./delta_center_corner_B3.png to _build/html/
exported in  _build/html/2408.09216.md
    + _build/html/tmp_2408.09216/./spectral_index_1x2.png
    + _build/html/tmp_2408.09216/./ring_corner_B3.png
    + _build/html/tmp_2408.09216/./delta_center_corner_B3.png
found figures ['tmp_2408.10199/./IRAS16547_Band6pa_debiased_3sig.png', 'tmp_2408.10199/./mom8-eps-converted-to-3.png', 'tmp_2408.10199/./figure_mom1_cs-2.png', 'tmp_2408.10199/./IRAS16547_Band6pol_debiased_2sig.png', 'tmp_2408.10199/./IRAS16547_Band6polfrac_debiased_2sig.png']
copying  tmp_2408.10199/./IRAS16547_Band6pa_debiased_3sig.png to _build/html/
copying  tmp_2408.10199/./mom8-eps-converted-to-3.png to _build/html/
copying  tmp_2408.1019

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Asymmetric dust accumulation of the PDS 70 disk revealed by ALMA Band 3 observations

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2408.09216-b31b1b.svg)](https://arxiv.org/abs/2408.09216)<mark>Appeared on: 2024-08-20</mark> -  _submitted_

</div>
<div id="authors">

K. D. (土井聖明), et al. -- incl., <mark>M. Benisty</mark>

</div>
<div id="abstract">

**Abstract:** The PDS 70 system, hosting two planets within its disk, is an ideal target for examining the effect of planets on dust accumulation, growth, and ongoing planet formation.Here, we present high-resolution ( $0\farcs07 = 8  \mathrm{au}$ ) dust continuum observations of the PDS 70 disk in ALMA Band 3 (3.0 mm).While previous Band 7 observations showed a dust ring with slight asymmetry, our Band 3 observations reveal a more prominent asymmetric peak in the northwest direction, where the intensity is 2.5 times higher than in other directions and the spectral index is at the local minimum with $\alpha_{\mathrm{SED}} \sim 2.2$ .This indicates that a substantial amount of dust is accumulated both radially and azimuthally in the peak.We also detect point-source emission around the stellar position in the Band 3 image, which is likely to be free-free emission.We constrain the eccentricity of the outer ring to be $e<0.04$ from the position of the central star and the outer ring.From the comparison with numerical simulations, we constrain the mass of PDS 70c to be less than 4.9 Jupiter masses if the gas turbulence strength $\alpha_{\mathrm{turb}} = 10^{-3}$ .Then, we discuss the formation mechanism of the disk structures and further planet formation scenarios in the disk.

</div>

<div id="div_fig1">

<img src="tmp_2408.09216/./spectral_index_1x2.png" alt="Fig2" width="100%"/>

**Figure 2. -** 
      Top: The radial profile of the intensity in Band 7 (blue) and Band 3 (orange).
      The dashed line represents the Band 3 profile scaled to match the peak intensity of Band 7 for comparison of the radial morphology.
      The Gaussian in the upper right corner shows the major and minor axes of the beam after deprojection.
      Bottom: The radial profile of the spectral index.
      The shaded area represents the $1\sigma$ error range due to thermal noise.
      The error bar in the lower right corner indicates the $1\sigma$ error range due to the absolute flux calibration uncertainty.
     (*fig:PDS70_radial*)

</div>
<div id="div_fig2">

<img src="tmp_2408.09216/./ring_corner_B3.png" alt="Fig6" width="100%"/>

**Figure 6. -** 
    The corner plot of MCMC fitting for the ring of PDS 70 in Band 3. The dashed lines in the top panels show the 16th, 50th, and 84th percentiles of the marginal distributions. The blue lines show the parameters that maximize the joint probability. (*fig:MCMC_B3_ring*)

</div>
<div id="div_fig3">

<img src="tmp_2408.09216/./delta_center_corner_B3.png" alt="Fig3" width="100%"/>

**Figure 3. -** 
    The corner plot of the difference in the center position between the ring and the central source in Band 3.
    It shows that the center of the ring and the central source are consistent.
     (*fig:MCMC_B3_center_shift*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2408.09216"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\dechms}[4]{#1^{\rm h}#2^{\rm m}#3\mbox{^{\rm s}\mskip-7.6mu. }#4}$
$\newcommand{\intdms}[3]{#1^{\circ}#2'#3"}$
$\newcommand{\decdms}[4]{-#1^{\circ}#2'#3\mbox{"\mskip-7.6mu. }#4}$</div>



<div id="title">

# Magnetic Fields in Massive Star-forming Regions (MagMaR) IV: Tracing the Magnetic Fields in the O-type protostellar system IRAS 16547$-$4247

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2408.10199-b31b1b.svg)](https://arxiv.org/abs/2408.10199)<mark>Appeared on: 2024-08-20</mark> -  _Accepted by the Astrophysical Journal, 13 pages_

</div>
<div id="authors">

L. A. Zapata, et al. -- incl., <mark>H. Beuther</mark>, <mark>S. Li</mark>

</div>
<div id="abstract">

**Abstract:** The formation of the massive stars, and in particular, the role that the magnetic fields play in their early evolutionary phase is still farfrom being completely understood. Here, we present Atacama Large Millimeter/Submillimeter Array (ALMA) 1.2 mm full polarizedcontinuum, and H $^{13}$ CO $^+$ (3 $-$ 2), CS(5 $-$ 4), and HN $^{13}$ C(3 $-$ 2) line observations with a high angular resolution( $\sim$ 0.4 $"$ or 1100 au). In the 1.2 mm continuum emission, we reveal a dusty envelope surrounding the massive protostars, IRAS16547-E and IRAS16547-W,with dimensions of $\sim$ 10,000 au. This envelope has a bi-conical structure likely carved by the powerful thermal radio jet present in region.The magnetic fields vectors follow very-well the bi-conical envelope. The polarization fraction is $\sim$ 2.0 \% in this region.Some of these vectors seem to converge to IRAS 16547-E, and IRAS 16547-W, the most massive protostars. Moreover, the velocityfields revealed from the spectral lines H $^{13}$ CO $^+$ (3 $-$ 2), and HN $^{13}$ C(3 $-$ 2) show velocity gradients with agood correspondence with the magnetic fields, that maybe are tracing the cavities of molecular outflows or maybe in some parts infall.We derived a magnetic field strength in some filamentary regions that goes from 2 to 6.1 mG.  We also find that the CS(5 $-$ 4) molecularline emission reveals multiple outflow cavities or bow-shocks with different orientations, some of which seem to follow the NW-SE radio thermal jet.

</div>

<div id="div_fig1">

<img src="tmp_2408.10199/./IRAS16547_Band6pa_debiased_3sig.png" alt="Fig4" width="100%"/>

**Figure 4. -** \scriptsize ALMA magnetic field vectors overlaid onto the resulting 1.2 mm continuum emission (Stokes I) from IRAS 16547$-$4247.
The white vector segments trace the polarization percentage.  The vector segment is displayed more or less per every beam,
using only data with a 2$\sigma$ cut in the debiasing process. The black contours are from $-$0.5, 0.5, 1, 5, 10, 20, 40, and 60 times 0.2 mJy Beam$^{-1}$ the RMS-noise of the image.
The red contours are from $-$5, 5, 10, 20, 40, 60, 80, and 180 times the RMS-noise of the
VLA 3.6 cm continuum emission, which is 20 $\mu$Jy Beam$^{-1}$.   The 3.6 cm continuum emission delineates the radio thermal jet  with  a NW-SE
orientation reported by [Garay, et. al (2003)](), [Rodríguez, Moran and Franco-Hernández (2008)](). The synthesized beam (half-power contour) of the ALMA continuum image is shown in the bottom-right corner.
The scale bar in the right represent the peak flux of the ALMA 1.2 mm continuum emission. The dashed lines trace more or less the bi-conical dusty envelope revealed in this study.
The white circle traces the primary beam or the FWHM of 22.7$"$. The yellow dots trace the positions of the sources reported in Table   \ref{table1}. (*fig:fig1*)

</div>
<div id="div_fig2">

<img src="tmp_2408.10199/./mom8-eps-converted-to-3.png" alt="Fig3.1" width="50%"/><img src="tmp_2408.10199/./figure_mom1_cs-2.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** \scriptsize Upper: ALMA moment 8 map of the CS(5$-$4) from IRAS 16547$-$4247 overlaid onto the resulting 1.2 mm continuum emission (grey contours),
and the VLA 3.6 cm continuum emission (white contours). The grey contours are from 50, 125, and 375 times 0.2 mJy Beam$^{-1}$.
The white contours are from $-$5, 5, 10, 20, 40, 60, 80, and 180 times the RMS-noise of the VLA 3.6 cm continuum emission, which is 20 $\mu$Jy Beam$^{-1}$.
The 3.6 cm continuum emission delineates a radio free-free jet  with NW-SE  orientation reported by [Garay, et. al (2003)](), [Rodríguez, Moran and Franco-Hernández (2008)]().
The synthesized beam (half-power contour) of the ALMA continuum image is shown in the bottom-right corner.
The scale bar in the right represents the peak flux of the ALMA CS line emission. The blue dashed curves trace more or less the multiple bow-shocks and outflow cavities revealed by this molecule.
 The white lines show that aperture angle of the outflow is more than 150$^\circ$ in the north and south sides.
 Lower: Same as the upper image but with the ALMA moment 1 map of the CS(5$-$4). The scale bar in the right represents the radial velocity of the ALMA CS line emission.
  The magenta dashed lines mark the orientation of the CO(3$-$2) ouflows reported by [Higuchi, Saigo and Chibueze (2015)]().
 (*fig:fig5*)

</div>
<div id="div_fig3">

<img src="tmp_2408.10199/./IRAS16547_Band6pol_debiased_2sig.png" alt="Fig5.1" width="50%"/><img src="tmp_2408.10199/./IRAS16547_Band6polfrac_debiased_2sig.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** \scriptsize Left Panel: ALMA Polarization Intensity map overlaid with the 1.2 mm continuum emission from IRAS 16547$-$4247.
Right Panel: Same as the left panel, but showing the ALMA Polarization Fraction. In both panels, the black contours are from $-$0.5, 0.5, 1, 5, 10, 20, 40,  and 60
times 0.2 mJy Beam$^{-1}$ the RMS-noise of the image.  The synthesized beam (half-power contour) of the ALMA continuum
 image is shown in the bottom-right corner. The scale bar in the right represents the de-biased polarization intensity, and polarization fraction.
 These maps were constructed with a threshold of 2$\sigma$ for the polarization.   (*fig:fig2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2408.10199"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

165  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
